<a href="https://colab.research.google.com/github/shoaibimt/CancerAI/blob/main/file_7_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Imports
## Data manipulation and EDA
import zipfile
import pandas as pd
import numpy as np
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
#!pip install padelpy

In [ ]:
## Molecular descriptors calculation
#from padelpy import padeldescriptor

## Machine learning models
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, classification_report


In [ ]:
# Parameters
maxdepth = 10
nestimators = 200
n_splits = 5
output_file = f'RandomForest_maxdepth{maxdepth}_nestimators{nestimators}.bin'

In [ ]:
#y_clas_pchembl_value = df6['pchembl_bin']

In [ ]:
pchembl_values = pd.read_csv('/content/drive/MyDrive/inhibitor_prediction/bcr_abl/chembl_data_retrieve/ML_model_training/chembl_bin_values.csv')

In [ ]:
pchembl_values

,pchembl_bin
0,0
1,0
2,0
3,0
4,0
...,...
393,1
394,1
395,0
396,1


In [ ]:
# Data preparation
## Load feature matrix - CDK fingerprint (dataset with the best performance)
cdk_fingerprint = pd.read_csv('/content/drive/MyDrive/inhibitor_prediction/bcr_abl/chembl_data_retrieve/ML_model_training/CDK_training.csv')

In [ ]:
cdk_fingerprint.shape

(398, 1025)

In [ ]:
cdk_fingerprint.head()

,Name,FP1,FP2,FP3,FP4,FP5,FP6,FP7,FP8,FP9,...,FP1015,FP1016,FP1017,FP1018,FP1019,FP1020,FP1021,FP1022,FP1023,FP1024
0,CHEMBL199723,1,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,1,0,1,0
1,CHEMBL381738,1,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,1,0,1,0
2,CHEMBL370800,1,0,0,0,0,0,0,0,0,...,0,1,1,0,1,0,1,0,1,0
3,CHEMBL436509,1,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,1,0,1,0
4,CHEMBL165,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0


In [ ]:
## Obtain feature matrix with low variance features
cdk_low_var = cdk_fingerprint.drop('Name', axis=1)
cdk_low_var


,FP1,FP2,FP3,FP4,FP5,FP6,FP7,FP8,FP9,FP10,...,FP1015,FP1016,FP1017,FP1018,FP1019,FP1020,FP1021,FP1022,FP1023,FP1024
0,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,1,0,1,0
1,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,1,0,1,0
2,1,0,0,0,0,0,0,0,0,0,...,0,1,1,0,1,0,1,0,1,0
3,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,1,0,1,0
4,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393,0,1,0,0,0,0,1,0,0,0,...,0,1,0,0,1,0,0,0,0,0
394,0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,1,0,0,0,0,0
395,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
396,0,0,0,0,0,0,0,0,0,0,...,1,1,1,0,1,0,0,0,0,0


In [ ]:
## Load y target variable - pchembl values
pchembl_values = pd.read_csv('/content/drive/MyDrive/inhibitor_prediction/bcr_abl/chembl_data_retrieve/ML_model_training/chembl_bin_values.csv')
pchembl_values

,pchembl_bin
0,0
1,0
2,0
3,0
4,0
...,...
393,1
394,1
395,0
396,1


In [ ]:
## Function to remove low variance features
def remove_low_variance(input_data, threshold=0.1):
    selection = VarianceThreshold(threshold)
    selection.fit(input_data)
    return input_data[input_data.columns[selection.get_support(indices=True)]]

In [ ]:
select = VarianceThreshold(0.2)
select.fit(cdk_low_var)
Descriptor_Name= cdk_low_var.columns[select.get_support(indices=True)]
Descriptor_Name

Index(['FP2', 'FP3', 'FP7', 'FP9', 'FP12', 'FP14', 'FP15', 'FP17', 'FP23',
       'FP34',
       ...
       'FP1003', 'FP1006', 'FP1008', 'FP1009', 'FP1011', 'FP1012', 'FP1016',
       'FP1017', 'FP1019', 'FP1023'],
      dtype='object', length=363)

In [ ]:
#cp col '/content/drive/MyDrive/inhibitor_prediction/bcr_abl/chembl_data_retrieve/testing_folder/'

In [ ]:
cdk_low_var = remove_low_variance(cdk_low_var, threshold=(.8*(1-.8)))
cdk_low_var

,FP2,FP3,FP7,FP9,FP12,FP14,FP15,FP17,FP22,FP23,...,FP1005,FP1006,FP1008,FP1009,FP1011,FP1012,FP1016,FP1017,FP1019,FP1023
0,0,0,0,0,0,1,1,1,1,0,...,1,0,0,1,1,0,1,0,1,1
1,0,0,0,0,0,1,1,1,1,0,...,1,0,0,1,1,0,1,0,1,1
2,0,0,0,0,0,1,1,1,1,1,...,1,0,0,1,1,0,1,1,1,1
3,0,0,0,0,0,1,1,1,1,0,...,1,0,1,1,1,0,0,0,1,1
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393,1,0,1,0,0,1,0,1,0,0,...,0,0,1,1,1,1,1,0,1,0
394,0,0,0,0,0,1,0,1,0,0,...,1,0,1,1,0,1,1,1,1,0
395,0,0,0,0,0,1,0,1,0,0,...,0,0,1,1,0,1,1,0,1,0
396,0,0,0,0,0,1,0,1,0,0,...,1,0,1,1,0,1,1,1,1,0


In [ ]:
cdk_low_var.to_csv('cdk_low_var.csv', index= False)

In [ ]:
cp cdk_low_var.csv '/content/drive/MyDrive/inhibitor_prediction/bcr_abl/chembl_data_retrieve/testing_folder/cdk_low_var.csv'

In [ ]:
pd.read_csv('cdk_low_var.csv')

,FP2,FP3,FP7,FP9,FP12,FP14,FP15,FP17,FP22,FP23,...,FP1005,FP1006,FP1008,FP1009,FP1011,FP1012,FP1016,FP1017,FP1019,FP1023
0,0,0,0,0,0,1,1,1,1,0,...,1,0,0,1,1,0,1,0,1,1
1,0,0,0,0,0,1,1,1,1,0,...,1,0,0,1,1,0,1,0,1,1
2,0,0,0,0,0,1,1,1,1,1,...,1,0,0,1,1,0,1,1,1,1
3,0,0,0,0,0,1,1,1,1,0,...,1,0,1,1,1,0,0,0,1,1
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393,1,0,1,0,0,1,0,1,0,0,...,0,0,1,1,1,1,1,0,1,0
394,0,0,0,0,0,1,0,1,0,0,...,1,0,1,1,0,1,1,1,1,0
395,0,0,0,0,0,1,0,1,0,0,...,0,0,1,1,0,1,1,0,1,0
396,0,0,0,0,0,1,0,1,0,0,...,1,0,1,1,0,1,1,1,1,0


In [ ]:
## Data split
X_train, X_test, y_train, y_test = train_test_split(cdk_low_var, pchembl_values,
                                                    test_size=0.2, random_state=42)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((318, 509), (80, 509), (318, 1), (80, 1))

In [ ]:
y_train

,pchembl_bin
3,0
18,0
376,1
248,1
177,0
...,...
71,1
106,1
270,0
348,0


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
rf_pc=RandomForestClassifier(random_state=42)

In [ ]:
param_grid = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,8,10],
    'criterion' :['gini', 'entropy']
}

In [ ]:
'''param_grid = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth' : [4,5,6,8,10],
}'''

"param_grid = { \n    'n_estimators': [100, 200, 300, 400, 500],\n    'max_depth' : [4,5,6,8,10],\n}"

In [ ]:
rfc_pc = GridSearchCV(estimator=rf_pc, param_grid=param_grid, cv= 5, n_jobs = -1)
rfc_pc.fit(X_train, y_train.values.ravel())

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [4, 5, 6, 8, 10],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [100, 200, 300, 400, 500]})

In [ ]:
rfc_pc.best_params_

{'criterion': 'gini',
 'max_depth': 4,
 'max_features': 'log2',
 'n_estimators': 100}

In [ ]:
rfc2=RandomForestClassifier(random_state=42, max_features='log2', n_estimators= 100, max_depth=4, criterion='gini', n_jobs =-1)
#rfc2=RandomForestClassifier(random_state=42, n_estimators= 400, max_depth=4, n_jobs =-1)

In [ ]:
rfc2.fit(X_train, y_train.values.ravel())

RandomForestClassifier(max_depth=4, max_features='log2', n_jobs=-1,
                       random_state=42)

In [ ]:
y_pred2 = rfc2.predict(X_test)
print("Accuracy_of_the_model :",accuracy_score(y_test,y_pred2)*100)
print("Auc_roc_of_the_model :",roc_auc_score(y_test,y_pred2))
print("Classification Report :")
print(classification_report(y_test,y_pred2))

Accuracy_of_the_model : 77.5
Auc_roc_of_the_model : 0.7916666666666667
Classification Report :
              precision    recall  f1-score   support

           0       0.67      0.88      0.76        32
           1       0.89      0.71      0.79        48

    accuracy                           0.78        80
   macro avg       0.78      0.79      0.77        80
weighted avg       0.80      0.78      0.78        80



In [ ]:
cv_scores = cross_val_score(rfc2, X_train, y_train.values.ravel(), cv=5, scoring="roc_auc")
cv_scores

array([0.67931034, 0.74285714, 0.82463054, 0.87755102, 0.8       ])

**diferent RF model**

In [ ]:
# Training
print(f'Training the model')

rf = RandomForestClassifier(max_depth=maxdepth, n_estimators=nestimators, n_jobs=-1,
                            random_state=42).fit(X_train, y_train.values.ravel())

Training the model


In [ ]:
# Validation

print(f'Doing validation')
print(f'ROC AUC is used as performance metric')

cv_scores = cross_val_score(rfc2, X_train, y_train.values.ravel(), cv=n_splits, scoring="roc_auc")

print('Validation results:')
print(f'RandomForest with max_depth={maxdepth} and n_estimators={nestimators}: \
    {np.mean(cv_scores):.2f} +- {np.std(cv_scores):.2f}')

Doing validation
ROC AUC is used as performance metric
Validation results:
RandomForest with max_depth=10 and n_estimators=200:     0.78 +- 0.07


In [ ]:
# Predicting values with RF model on test dataset
print(f'Doing evaluation on test dataset')
y_test_pred = rfc2.predict(X_test)

Doing evaluation on test dataset


In [ ]:
## Calculate performance metrics
roc_auc_test = roc_auc_score(y_test, y_test_pred)

acc_test = accuracy_score(y_test, y_test_pred)

prec_test = precision_score(y_test, y_test_pred)

recall_test = recall_score(y_test, y_test_pred)

f1score_test = f1_score(y_test, y_test_pred)

In [ ]:
# Print metrics values
print(f'Testing results:')
metrics_names = ["ROC_AUC", "Accuracy", "Precision", "Recall", "F1 score"]
metrics_values = [roc_auc_test, acc_test, prec_test, recall_test, f1score_test]

Testing results:


In [ ]:
metrics = dict(zip(metrics_names, metrics_values))

In [ ]:
headers = ('Metric', 'Value')
spaces = ('----------', '----------')

In [ ]:
head_spaces = [headers, spaces]


In [ ]:
for metric, value in head_spaces:
    print(f'{metric:>10s} {value:>10s}')

    Metric      Value
---------- ----------


In [ ]:
for metric, value in metrics.items():
    print(f'{metric:>10s} {value:>10.2f}')

   ROC_AUC       0.79
  Accuracy       0.78
 Precision       0.89
    Recall       0.71
  F1 score       0.79


In [ ]:
# Save the model

with open(output_file, "wb") as f_out:
    pickle.dump(rfc2, f_out)

print(f'The model is saved to {output_file}')

The model is saved to RandomForest_maxdepth10_nestimators200.bin


In [ ]:
cp RandomForest_maxdepth10_nestimators200.bin '/content/drive/MyDrive/ML_model/'